In [35]:
import pdfplumber

with pdfplumber.open("bank_statement.pdf") as pdf:
    first_page = pdf.pages[0]
    # word_text = first_page.extract_text(
    #     x_tolerance=3, x_tolerance_ratio=None, y_tolerance=3, layout=False, x_density=7.25, y_density=13, line_dir_render=None, char_dir_render=None)
    # print(word_text)
    # Visualize the words on the first page

    im = first_page.to_image(resolution=150)
    words = first_page.extract_words(x_tolerance=0.1,  # Allow small spacing between characters in the same word
    y_tolerance=2,    # Allow small vertical shifts in the same line
    keep_blank_chars=True  # Ignore big spaces between words
 )
    
    print(words)    
    
    for word in words:
        im.draw_rect((word["x0"], word["top"], word["x1"], word["bottom"]), stroke_width=1)
    im.show()

[{'text': 'Transaction Details', 'x0': 54.0, 'x1': 147.60782501367578, 'top': 1.2862182000000075, 'doctop': 1.2862182000000075, 'bottom': 13.283293200000003, 'upright': True, 'height': 11.997074999999995, 'width': 93.60782501367578, 'direction': 'ltr'}, {'text': 'Page 1 of 19', 'x0': 727.3605195, 'x1': 787.6770675, 'top': 1.2862182000000075, 'doctop': 1.2862182000000075, 'bottom': 13.283293200000003, 'upright': True, 'height': 11.997074999999995, 'width': 60.31654800000001, 'direction': 'ltr'}, {'text': '05-04-2025 14:59:15                                     BANK OF INDIA, PALUS                                                  Page 1', 'x0': 59.458669125, 'x1': 780.0038201325007, 'top': 55.20572497500007, 'doctop': 55.20572497500007, 'bottom': 64.30351822500006, 'upright': True, 'height': 9.097793249999995, 'width': 720.5451510075006, 'direction': 'ltr'}, {'text': 'REP27', 'x0': 54.0, 'x1': 81.29337975, 'top': 65.51120197500006, 'doctop': 65.51120197500006, 'bottom': 74.60899522500006

In [ ]:
from collections import defaultdict


lines = defaultdict(list)

# Make sure word_text contains dicts, not just strings
for word in words:
    if isinstance(word, dict) and 'top' in word:
        y = round(word['top'], 1)
        lines[y].append((word['x0'], word['text']))

segments = []
for y in sorted(lines):
    
    line = sorted(lines[y])  # sort left to right
    segments.append([text for _, text in line])

for line in segments:
    print(line)
 
# print(lines)

['Transaction Details', 'Page 1 of 19']
['05-04-2025 14:59:15                                     BANK OF INDIA, PALUS                                                  Page 1']
['REP27']
['                                                       MAULI GARMENTSREGISTER                                                       ']
['------------------------------------------------------------------------------------------------------------------------------------']
['Report To               :M']
['Service OutLet          :16150   PALUS']
['Account Number          :161530110000120/INR       MAULI GARMENTS']
['Report for the Period   :01-04-2024TO31-03-2025']
['------------------------------------------------------------------------------------------------------------------------------------']
['05-04-2025 14:59:15                                     BANK OF INDIA, PALUS                                                  Page 2']
['REP27']
['                                                       MA

In [48]:
import re

acc_number_pattern = r"Account Number\s*:([0-9]+(?:/[A-Z]+)?)"
report_period_pattern = r"Report for the Period\s*:(\d{2}-\d{2}-\d{4})TO(\d{2}-\d{2}-\d{4})"
opening_balance_pattern = re.compile(
    r"Account\s+Opening\s+balance\s*:\s*(\d{1,3}(?:,\d{3})*|\d+)\.\d{2}(DR|CR)"
)

bank_name_pattern = re.compile(
    r"(?i)\b(?:[A-Z&]{2,}\s+)*BANK(?:\s+[A-Z&]{2,})*\b(?:,\s*\w+)?"
)

txn_pattern = re.compile(
    r"^\d{2}-\d{2}-\d{4}[A-Z0-9 ]{8,}\s+.*\d{1,3}(?:,\d{3})*\.\d{2}\s*(?:\d{1,3}(?:,\d{3})*\.\d{2}\s*)?(DR|CR)$"
)

account_number = None
report_period = None
normalized_amount = None
direction = None
Bank_Name = None
transactions = []

for line in segments:
    line_str = " ".join(line).strip()
    
    # Bank Name
    match = bank_name_pattern.search(line_str)
    if match:
        Bank_Name = match.group().strip()
        

    # Account Number
    acc_match = re.search(acc_number_pattern, line_str)
    if acc_match:
        account_number = acc_match.group(1)
        
    # Opening Balance
    match = opening_balance_pattern.search(line_str)
    if match:
        amount = match.group(1).replace(',', '')  # Remove commas
        direction = match.group(2)                # DR or CR
        normalized_amount = float(amount)
        if direction == "DR":
            normalized_amount *= -1

    # Report Period
    period_match = re.search(report_period_pattern, line_str)
    if period_match:
        report_period = (period_match.group(1), period_match.group(2))

    # Transactions
    if txn_pattern.match(line_str):
        transactions.append(line_str)
  
print("Bank Name:", Bank_Name)      
print("Account Number:", account_number)
print("Report Period:", report_period)
print("Opening Balance:", normalized_amount, "Direction:", direction)
print("Transactions:")
for txn in transactions:
    print(txn)


Bank Name: BANK OF INDIA, PALUS
Account Number: 161530110000120/INR
Report Period: ('01-04-2024', '31-03-2025')
Opening Balance: -989057.0 Direction: DR
Transactions:
03-04-2024S30619487            UPI/409426505852/CR/KIRAN                               4,000.00         9,85,057.13DR
03-04-2024S43261464 TCS/409421 181892174789//SOL/152.59.                               7,000.00         9,78,057.13DR
04-04-2024S55441873            NEW SUNIL CAP                      10,000.00                             9,88,057.13DR
06-04-2024S23572980 0000004016 Cash dep at R1615030                                   10,000.00         9,78,057.13DR
06-04-2024S24949925            StCon-36978549/Sai Hosier          10,000.00                             9,88,057.13DR
07-04-2024S27618422 0000000012 EDCPOS Credit / MAULI GAR                               2,200.00         9,85,857.13DR
08-04-2024S72712956 TCS/409920 224610558590//SOL/152.58.                              10,000.00         9,75,857.13DR
09-04-2

In [80]:
import pandas as pd
import re


pattern = re.compile(
    r"(?P<date>\d{2}-\d{2}-\d{4})\s?"                  # Date
    r"(?P<tran_id>[A-Z0-9]{6,10})\s+"                    # Transaction ID
    r"(?:(?P<ref>[A-Za-z0-9][A-Za-z0-9/._-]{8}\d)\s)?"                        # Optional Ref Num
    r"(?P<part>.*?)"                                # Particulars (non-greedy)
    r"(?P<amt1>\d{1,3}(?:,\d{2,3})*\.\d{2})?"        # First amount (Debit or Credit)
    r"\s*"                                          
    r"(?P<amt2>\d{1,3}(?:,\d{2,3})*\.\d{2})"         # Second amount (always Balance)
    r"(?P<type>(?i:CR|DR))"                         # DR/CR at the end
)


rows = []

for line in transactions:
    match = pattern.search(line)
    if match:
        d = match.groupdict()
        date = d['date']
        tran = d['tran_id']
        ref = d['ref'].strip() if d['ref'] else ""
        part = d['part'].strip()
        amt1 = d['amt1'].replace(",", "") if d['amt1'] else ""
        amt2 = d['amt2'].replace(",", "") if d['amt2'] else ""
        typ = d['type'].upper()

        # Determine debit/credit using distance logic
        debit = credit = ""
        if amt1:
            # Estimate spacing between particulars and amounts
            part_end_idx = line.find(d['part']) + len(d['part'])
            amt1_idx = line.find(d['amt1'], part_end_idx)
            amt2_idx = line.find(d['amt2'], amt1_idx)

            if amt1_idx - part_end_idx < 20:  # If amt1 is close to particulars
                debit = amt1
            else:
                credit = amt1
        
        # amt2 is always balance
        balance = amt2

        rows.append({
            "Date": date,
            "Tran ID": tran,
            "Ref Num": ref,
            "Particulars": part,
            "Debit Amt.": debit if typ == "DR" else "",
            "Credit Amt.": credit if typ == "CR" else "",
            "Balance Amt.": balance,
            "Type": typ
        })

# Create DataFrame
df = pd.DataFrame(rows)

# Convert to numeric values
for col in ["Debit Amt.", "Credit Amt.", "Balance Amt."]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df


,Date,Tran ID,Ref Num,Particulars,Debit Amt.,Credit Amt.,Balance Amt.,Type
0,03-04-2024,S30619487,,UPI/409426505852/CR/KIRAN,4000.0,NaN,985057.13,DR
1,03-04-2024,S43261464,TCS/409421,181892174789//SOL/152.59.,7000.0,NaN,978057.13,DR
2,04-04-2024,S55441873,,NEW SUNIL CAP,10000.0,NaN,988057.13,DR
3,06-04-2024,S23572980,0000004016,Cash dep at R1615030,10000.0,NaN,978057.13,DR
4,06-04-2024,S24949925,,StCon-36978549/Sai Hosier,10000.0,NaN,988057.13,DR
5,07-04-2024,S27618422,0000000012,EDCPOS Credit / MAULI GAR,2200.0,NaN,985857.13,DR
6,08-04-2024,S72712956,TCS/409920,224610558590//SOL/152.58.,10000.0,NaN,975857.13,DR
7,09-04-2024,S87582618,0005670449,IBNEFT/HDFC/Sonima garmen,10000.0,NaN,985857.13,DR
8,09-04-2024,S88435601,,RAJASTHAN HOSIERY,4000.0,NaN,989857.13,DR
9,12-04-2024,S66907478,0000004015,Cash dep at R1615030,12000.0,NaN,977857.13,DR


In [109]:
import pdfplumber
import pandas as pd
import re
from collections import defaultdict
from typing import List, Dict, Tuple, Optional

class BankStatementExtractor:
    """
    A class to extract and parse bank statement data from PDF files.
    """
    
    def __init__(self):
        # Regex patterns for extracting different components
        self.unmatched_lines = 0
        self.unmatched_lines_no = []
        self.patterns = {
            'account_number': re.compile(r"Account Number\s*:([0-9]+(?:/[A-Z]+)?)"),
            'report_period': re.compile(r"Report for the Period\s*:(\d{2}-\d{2}-\d{4})TO(\d{2}-\d{2}-\d{4})"),
            'opening_balance': re.compile(r"Account\s+Opening\s+balance\s*:\s*(\d{1,3}(?:,\d{3})*|\d+)\.\d{2}(DR|CR)"),
            'bank_name': re.compile(r"(?i)\b(?:[A-Z&]{2,}\s+)*BANK(?:\s+[A-Z&]{2,})*\b(?:,\s*\w+)?"),
            'transaction': re.compile(r"^\d{2}-\d{2}-\d{4}[A-Z0-9 ]{8,}\s+.*\d{1,3}(?:,\d{3})*\.\d{2}\s*(?:\d{1,3}(?:,\d{3})*\.\d{2}\s*)?(DR|CR)$"),
            'transaction_detail': re.compile(
                    r"(?P<date>\d{2}-\d{2}-\d{4})\s*"                  # Date
                    r"(?P<tran_id>[A-Z0-9]{6,10})\s+"                    # Transaction ID
                    r"(?:(?P<ref>[A-Za-z0-9][A-Za-z0-9/._-]{8}\d)\s)?"                        # Optional Ref Num
                    r"(?P<part>.*?)"                                # Particulars (non-greedy)
                    r"(?P<amt1>\d{1,3}(?:,\d{2,3})*\.\d{2})?"        # First amount (Debit or Credit)
                    r"\s*"                                          
                    r"(?P<amt2>\d{1,3}(?:,\d{2,3})*\.\d{2})"         # Second amount (always Balance)
                    r"(?P<type>(?i:CR|DR))" 
            )
        }
    
    def extract_text_from_pdf(self, pdf_path: str) -> List[List[str]]:
        """
        Extract text from PDF and organize into lines based on y-coordinates.
        
        Args:
            pdf_path (str): Path to the PDF file
            
        Returns:
            List[List[str]]: List of lines, where each line is a list of words
        """
        all_words = []
        
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                words = page.extract_words(
                    x_tolerance=0.1,      # Tolerance for character merging horizontally
                    y_tolerance=2,        # Tolerance for slight vertical drift
                    keep_blank_chars=True # Preserve spaces within words
                )
                all_words.append(words)
        
        lines_per_page = []
        # Group words by their y-coordinate (line)
        for page in all_words:
            
            lines = defaultdict(list)
            
            # Iterate through words and group them by their y-coordinate
            for word in page:
                if isinstance(word, dict) and 'top' in word:
                    y = round(word['top'], 1)
                    lines[y].append((word['x0'], word['text']))
        
            # Sort lines by y-coordinate and words within each line by x-coordinate
            page_segments = []
            for y in sorted(lines):
                line = sorted(lines[y])  # Sort left to right
                page_segments.append([text for _, text in line])
        
            lines_per_page.append(page_segments)
        
        
        return lines_per_page
    
    def extract_metadata(self, lines_per_page: List[List[str]]) -> Dict:
            """
            Extract metadata from the bank statement (account number, bank name, etc.).
            
            Args:
                segments (List[List[str]]): List of text lines
                
            Returns:
                Dict: Dictionary containing extracted metadata
            """
            metadata = {
                'bank_name': None,
                'account_number': None,
                'report_period': None,
                'opening_balance': None,
                'opening_balance_type': None
            }
            
            for page in lines_per_page:
                for line in page:
                    line_str = " ".join(line).strip()
                    
                    # Extract bank name
                    if not metadata['bank_name']:
                        match = self.patterns['bank_name'].search(line_str)
                        if match:
                            metadata['bank_name'] = match.group().strip()
                    
                    # Extract account number
                    if not metadata['account_number']:
                        match = self.patterns['account_number'].search(line_str)
                        if match:
                            metadata['account_number'] = match.group(1)
                    
                    # Extract report period
                    if not metadata['report_period']:
                        match = self.patterns['report_period'].search(line_str)
                        if match:
                            metadata['report_period'] = (match.group(1), match.group(2))
                    
                    # Extract opening balance
                    if not metadata['opening_balance']:
                        match = self.patterns['opening_balance'].search(line_str)
                        if match:
                            amount = float(match.group(1).replace(',', ''))
                            direction = match.group(2)
                            metadata['opening_balance'] = amount if direction == "CR" else -amount
                            metadata['opening_balance_type'] = direction
                    
                    if all([
                                metadata['bank_name'],
                                metadata['account_number'],
                                metadata['report_period'],
                                metadata['opening_balance']
                            ]):
                        break  # Break inner loop
                else:
                    continue # Continue outer loop if inner loop didn't break
                break # Break outer loop if all metadata is found
                
            return metadata
        
    def extract_transactions(self, lines_per_page: List[List[str]]) -> List[str]:
        """
        Extract transaction lines from the segments.
        
        Args:
            segments (List[List[str]]): List of text lines
            
        Returns:
            List[str]: List of transaction strings
        """
        transactions = []
        for page in lines_per_page:
            for line in page:
                line_str = " ".join(line).strip()
                if self.patterns['transaction'].match(line_str):
                    transactions.append(line_str)
        return transactions
    
    def parse_transactions_to_dataframe(self, raw_lines: List[str]) -> pd.DataFrame:
        """
        Parse a list of raw transaction lines and return a structured pandas DataFrame.
        
        Args:
            raw_lines (List[str]): List of raw transaction strings
            
        Returns:
            pd.DataFrame: Structured transaction data
        """
        transactions = []
        line_no = 0  # Initialize line number for tracking unmatched lines

        for line in raw_lines:
            line_no += 1  # Increment line number for each line processed
            match = self.patterns['transaction_detail'].search(line)
            if not match:
                self.unmatched_lines_no.append(line_no)  # Track unmatched lines
                self.unmatched_lines += 1
                
                continue  # Skip lines that don't match
            
            data = match.groupdict()

            parsed = {
                'date': data['date'],
                'transaction_id': data['tran_id'],
                'reference_number': data['ref'].strip() if data['ref'] else "",
                'particulars': data['part'].strip(),
                'debit_amount': None,
                'credit_amount': None,
                'balance_amount': float(data['amt2'].replace(",", "")),
                'type': data['type'].upper()
            }

            if data['amt1']:
                amt1_value = float(data['amt1'].replace(",", ""))
                part_end_idx = line.find(data['part']) + len(data['part'])
                amt1_idx = line.find(data['amt1'], part_end_idx)
                is_debit_position = (amt1_idx - part_end_idx) < 20

                if parsed['type'] == 'DR':
                    parsed['debit_amount'] = amt1_value
                else:
                    parsed['credit_amount'] = amt1_value

            transactions.append(parsed)

        # Return early if empty
        if not transactions:
            return pd.DataFrame(columns=[
                'Date', 'Transaction ID', 'Reference Number', 'Particulars',
                'Debit Amount', 'Credit Amount', 'Balance Amount', 'Type'
            ])

        # Convert to DataFrame
        df = pd.DataFrame(transactions)

        # Rename columns
        df = df.rename(columns={
            'date': 'Date',
            'transaction_id': 'Transaction ID',
            'reference_number': 'Reference Number',
            'particulars': 'Particulars',
            'debit_amount': 'Debit Amount',
            'credit_amount': 'Credit Amount',
            'balance_amount': 'Balance Amount',
            'type': 'Type'
        })

        # Parse date
        df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')

        # Cast numeric types
        for col in ['Debit Amount', 'Credit Amount', 'Balance Amount']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        return df

    def process_bank_statement(self, pdf_path: str, debug: bool = False) -> Tuple[Dict, pd.DataFrame]:
        """
        Complete pipeline to process a bank statement PDF.
        
        Args:
            pdf_path (str): Path to the PDF file
            debug (bool): Enable debug output to help troubleshoot
            
        Returns:
            Tuple[Dict, pd.DataFrame]: Metadata and transaction DataFrame
        """
        print("Extracting text from PDF...")
        lines_per_page = self.extract_text_from_pdf(pdf_path)
        if not segments:
            raise ValueError("No text extracted from the PDF. Please check the file path or content.")
        metadata = self.extract_metadata(lines_per_page)
        transaction_lines = self.extract_transactions(lines_per_page)
        df = self.parse_transactions_to_dataframe(transaction_lines)
        
        
        if debug:
            print("Debug mode enabled.")
        #     print(f"\n=== DEBUG: First 10 lines ===")
        #     for i, segment in enumerate(segments[:10]):
        #         line_str = " ".join(segment).strip()
        #         print(f"Line {i}: {line_str}")
        
        # print("Extracting metadata...")
        # metadata = self.extract_metadata(segments)
        
        # print("Extracting transactions...")
        # transaction_lines = self.extract_transactions(segments)
        
        # if debug:
        #     print(f"\n=== DEBUG: Transaction lines found ===")
        #     for i, line in enumerate(transaction_lines[:5]):
        #         print(f"Transaction {i}: {line}")
        
        # print(f"Found {len(transaction_lines)} transactions. Parsing...")
        # parsed_transactions = []
        # for line in transaction_lines:
        #     parsed = self.parse_transaction(line)
        #     if parsed:
        #         parsed_transactions.append(parsed)
        #     elif debug:
        #         print(f"Failed to parse: {line}")
        
        # print("Creating DataFrame...")
        # df = self.create_dataframe(parsed_transactions)
        
        # print("Processing complete!")
        return metadata, df, self.unmatched_lines, self.unmatched_lines_no
    
extractor = BankStatementExtractor()
    
pdf_path = "bank_statement.pdf"  # Replace with your PDF file path
    
# segments = extractor.extract_text_from_pdf(pdf_path)
# # print(f"Extracted {len(segments)} lines from the PDF.")
# # for line in segments:
# #     print(line)
    
# metadata = extractor.extract_metadata(segments)
# print("Extracted Metadata:")
# for key, value in metadata.items():
#     print(f"{key}: {value}")
    
# transactions = extractor.extract_transactions(segments)
# print("\nExtracted Transactions:")
# for txn in transactions:
#     print(txn)
# print(f"Total transactions extracted: {len(transactions)}")

# df = extractor.parse_transactions_to_dataframe(transactions)
# print("\nParsed Transactions DataFrame:")

# print(f"Total unmatched lines: {extractor.unmatched_lines}")
# print(f"list of unmatched lines: {extractor.unmatched_lines_no}")

m, d , u, u_no = extractor.process_bank_statement(pdf_path, debug=False) 
print("\nMetadata:")
for key, value in m.items():
    print(f"{key}: {value}")
print("\nTransactions DataFrame:")
print(d)
print(f"Total unmatched lines: {u}")
print(f"List of unmatched lines: {u_no}")
print(f"length of DataFrame: {len(d)}")
# print("\nProcessing complete!")

print(d)

Extracting text from PDF...

Metadata:
bank_name: BANK OF INDIA, PALUS
account_number: 161530110000120/INR
report_period: ('01-04-2024', '31-03-2025')
opening_balance: -989057.0
opening_balance_type: DR

Transactions DataFrame:
          Date Transaction ID Reference Number                Particulars  \
0   2024-04-03      S30619487                   UPI/409426505852/CR/KIRAN   
1   2024-04-03      S43261464       TCS/409421  181892174789//SOL/152.59.   
2   2024-04-04      S55441873                               NEW SUNIL CAP   
3   2024-04-06      S23572980       0000004016       Cash dep at R1615030   
4   2024-04-06      S24949925                   StCon-36978549/Sai Hosier   
..         ...            ...              ...                        ...   
677 2025-03-31      S78582212                   UPI/193706630611/CR/Mrs S   
678 2025-03-31      S79127882                   StCon-41580538/Mauli garm   
679 2025-03-31      S79241029                   UPI/509012280710/CR/Mrs P   
68

In [1]:
import pdfplumber

with pdfplumber.open("kotak.pdf", password="619132316") as pdf:
    first_page = pdf.pages[0]
    # word_text = first_page.extract_text(
    #     x_tolerance=3, x_tolerance_ratio=None, y_tolerance=3, layout=False, x_density=7.25, y_density=13, line_dir_render=None, char_dir_render=None)
    # print(word_text)
    # Visualize the words on the first page

    im = first_page.to_image(resolution=150)
    words = first_page.extract_words(x_tolerance=0.1,  # Allow small spacing between characters in the same word
    y_tolerance=2,    # Allow small vertical shifts in the same line
    keep_blank_chars=True  # Ignore big spaces between words
 )
    
    print(words)    
    
    for word in words:
        im.draw_rect((word["x0"], word["top"], word["x1"], word["bottom"]), stroke_width=1)
    im.show()

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


[{'text': 'MANGESH NITIN SALUNKE', 'x0': 37.5, 'x1': 143.85075, 'top': 41.27025000000003, 'doctop': 41.27025000000003, 'bottom': 49.52025000000003, 'upright': True, 'height': 8.25, 'width': 106.35075, 'direction': 'ltr'}, {'text': 'Period', 'x0': 325.3125, 'x1': 349.155, 'top': 41.27025000000003, 'doctop': 41.27025000000003, 'bottom': 49.52025000000003, 'upright': True, 'height': 8.25, 'width': 23.842499999999973, 'direction': 'ltr'}, {'text': ':', 'x0': 406.16251, 'x1': 408.45601, 'top': 41.27025000000003, 'doctop': 41.27025000000003, 'bottom': 49.52025000000003, 'upright': True, 'height': 8.25, 'width': 2.2934999999999945, 'direction': 'ltr'}, {'text': '01-11-2024 to 30-04-2025', 'x0': 417.375, 'x1': 513.2235000000001, 'top': 41.27025000000003, 'doctop': 41.27025000000003, 'bottom': 49.52025000000003, 'upright': True, 'height': 8.25, 'width': 95.84850000000006, 'direction': 'ltr'}, {'text': 'Cust Reln No', 'x0': 325.3125, 'x1': 374.367, 'top': 59.08275000000003, 'doctop': 59.08275000